In [1]:
import pandas as pd

In [6]:
products = pd.read_csv('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/products_export.csv')
reviews = pd.read_csv('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/reviews_export.csv')
weighted_trait_df_heatmap = pd.read_csv('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/weighted_trait_heatmap.csv')
# weighted_trait_df_graph = pd.read_csv('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/weighted_trait_graph_export.csv')
cluster_solutions_df = pd.read_csv('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/cluster_solutions.csv')

print(weighted_trait_df_heatmap.columns)
print(products.columns)
print(reviews.columns)
print(weighted_trait_df_heatmap.columns)
print(cluster_solutions_df.columns)

# Assume that 'products', 'reviews', 'weighted_trait_df_graph', 'cluster_solutions_df' are your dataframes.

# Let's merge 'products' and 'reviews' on 'asin' column (which is Amazon's unique identifier for products).
merged_df = pd.merge(products, reviews, on='asin')
weighted_trait_df_heatmap['id'] = weighted_trait_df_heatmap['id'].map(eval)

# Expand list 'id' values into multiple rows
weighted_trait_df_heatmap = weighted_trait_df_heatmap.explode('id')

# Convert the 'id' column in both dataframes to int
merged_df['id'] = merged_df['id'].astype(int)

weighted_trait_df_heatmap['id'] = weighted_trait_df_heatmap['id'].astype(int)

# Now, you can merge
merged_df = pd.merge(merged_df, weighted_trait_df_heatmap, on='id', how='left', suffixes=('', '_y'))

print('cluster_label' in merged_df.columns)
print('cluster_label' in cluster_solutions_df.columns)

# If 'cluster_label' in 'cluster_solutions_df' matches 'cluster_label' in 'weighted_trait_df_graph', let's merge these dataframes.
merged_df = pd.merge(merged_df, cluster_solutions_df, on='cluster_label', how='left', suffixes=('', '_z'))

# Remove duplicate columns after merging
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]

# Create quantifiable observations
# Count of reviews per product
reviews_per_product = merged_df['asin'].value_counts()

# Average rating per product
average_rating_per_product = merged_df.groupby('asin')['rating'].mean()

# Count of reviews per rating
reviews_per_rating = merged_df['rating'].value_counts()

# Count of reviews per user description
reviews_per_user_description = merged_df['user_description'].value_counts()

# Average positive sentiment per product
average_positive_sentiment_per_product = merged_df.groupby('asin')['positive_sentiment'].mean()

# Average negative sentiment per product
average_negative_sentiment_per_product = merged_df.groupby('asin')['negative_sentiment'].mean()

Index(['asin', 'data_label', 'positive_sentiment', 'negative_sentiment',
       'rating_avg', 'rating', 'id', 'how_product_is_used', 'media',
       'where_product_is_used', 'user_description', 'observation_count',
       'percentage', 'type', 'cluster_label'],
      dtype='object')
Index(['index', 'title', 'description', 'feature_bullets', 'variants',
       'categories', 'asin', 'url', 'reviews_total_reviews', 'reviews_rating',
       'reviews_answered_questions', 'item_available', 'price_symbol',
       'price_currency', 'price_current_price', 'price_discounted',
       'price_before_price', 'price_savings_amount', 'price_savings_percent',
       'bestsellers_rank', 'main_image', 'total_images', 'images',
       'total_videos', 'videos', 'delivery_message',
       'product_information_dimensions', 'product_information_weight',
       'product_information_available_from',
       'product_information_available_from_utc',
       'product_information_available_for_months',
       'produ

In [8]:
merged_df.columns

Index(['index', 'title_x', 'description', 'feature_bullets', 'variants',
       'categories', 'asin', 'url', 'reviews_total_reviews', 'reviews_rating',
       'reviews_answered_questions', 'item_available', 'price_symbol',
       'price_currency', 'price_current_price', 'price_discounted',
       'price_before_price', 'price_savings_amount', 'price_savings_percent',
       'bestsellers_rank', 'main_image', 'total_images', 'images',
       'total_videos', 'videos', 'delivery_message',
       'product_information_dimensions', 'product_information_weight',
       'product_information_available_from',
       'product_information_available_from_utc',
       'product_information_available_for_months',
       'product_information_available_for_days',
       'product_information_manufacturer', 'product_information_model_number',
       'product_information_department', 'product_information_qty_per_order',
       'product_information_store_id', 'product_information_brand',
       'badges_amazon

In [ ]:
# Create a new dataframe with all these quantifiable data
quantifiable_data = pd.DataFrame({
    'reviews_per_product': reviews_per_product,
    'average_rating_per_product': average_rating_per_product,
    'reviews_per_rating': reviews_per_rating,
    'reviews_per_user_description': reviews_per_user_description,
    'average_positive_sentiment_per_product': average_positive_sentiment_per_product,
    'average_negative_sentiment_per_product': average_negative_sentiment_per_product,
})

# Reset the index
quantifiable_data = quantifiable_data.reset_index()


quantifiable_data.head(40)

In [15]:
# Read the traits information from the database
weighted_trait_df_graph = pd.read_csv('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/weighted_trait_graph_export.csv')

weighted_trait_df_graph['id'] = weighted_trait_df_graph['id'].map(eval)

# Function to concatenate the lists but keep only distinct elements
def unique_elements(lst):
    return list(set([i for sublist in lst for i in sublist]))

# Group the DataFrame by 'cluster_label', apply the function to 'id_list'
grouped_df = weighted_trait_df_graph.groupby('cluster_label').agg({'id': unique_elements}).reset_index()

# Merge this DataFrame with the original one to create the 'cluster_id_list' column
weighted_trait_df_graph = pd.merge(weighted_trait_df_graph, grouped_df, how="left", on="cluster_label")
weighted_trait_df_graph.rename(columns={"id_y": "cluster_id_list", "id_x":"id"}, inplace=True)

In [38]:
weighted_trait_df_graph.columns

Index(['data_label', 'positive_sentiment', 'negative_sentiment', 'rating_avg',
       'rating', 'id', 'how_product_is_used', 'media', 'where_product_is_used',
       'user_description', 'asin', 'observation_count', 'type',
       'cluster_label', 'cluster_id_list'],
      dtype='object')

In [17]:
weighted_trait_df_graph.drop(columns =['cluster_problem_statement',
       'cluster_solution_1_title', 'cluster_solution_1_description',
       'cluster_solution_2_title', 'cluster_solution_2_description',
       'cluster_solution_3_title', 'cluster_solution_3_description', 'solutions'], inplace = True )

In [19]:
weighted_trait_df_graph.columns

Index(['data_label', 'positive_sentiment', 'negative_sentiment', 'rating_avg',
       'rating', 'id', 'how_product_is_used', 'media', 'where_product_is_used',
       'user_description', 'asin', 'observation_count', 'type',
       'cluster_label', 'cluster_id_list'],
      dtype='object')

In [20]:
cluster_solutions_df.columns

Index(['cluster_label', 'type', 'cluster_problem_statement',
       'cluster_solution_title', 'cluster', 'improvement_title',
       'improvement_objective', 'improvement_solution', 'improvement_steps',
       'improvement_considerations'],
      dtype='object')

In [14]:
cluster_solutions_df[cluster_solutions_df['cluster_label'] == 'Sensory Toys with Balls and Beads']

,cluster_label,type,cluster_problem_statement,cluster_solution_title,cluster,improvement_title,improvement_objective,improvement_solution,improvement_steps,improvement_considerations
5,Sensory Toys with Balls and Beads,Fact,The product has received multiple complaints r...,Reducing the Noise Level of the Magnetic Beads...,0,Noise Reduction and Sound Dampening,Reduce the noise level of the product to enhan...,Add a volume control feature to allow users to...,Volume Control: Add a control knob on the side...,Conduct a feasibility analysis for cost-effect...
28,Sensory Toys with Balls and Beads,Fact,The product has received multiple complaints r...,Reducing the Noise Level of the Magnetic Beads...,5,Improved Pen Attachment and Storage Design,To address issues related to the pen's quality...,Incorporate a mechanism into the product desig...,Conduct a thorough analysis to determine the m...,Ensure that the mechanism is user-friendly and...
31,Sensory Toys with Balls and Beads,Fact,The product has received multiple complaints r...,Reducing the Noise Level of the Magnetic Beads...,7,Improved Magnetic Ball Stability,Address the issue of magnetic balls falling of...,Increase the magnet strength to ensure that th...,Magnet Strength: Source and test different typ...,Ensure that the increased magnet strength does...
46,Sensory Toys with Balls and Beads,Fact,The product has received multiple complaints r...,Reducing the Noise Level of the Magnetic Beads...,0,Enhanced Ball Mechanism and Magnet Strength,Improve the ball mechanism and magnet strength...,"Use a quieter ball mechanism, such as a foam b...",Ball Mechanism: Review and redesign the ball m...,Ensure the product meets safety standards and ...
69,Sensory Toys with Balls and Beads,Fact,The product has received multiple complaints r...,Reducing the Noise Level of the Magnetic Beads...,5,Enhanced Magnetic Functionality and Noise Redu...,To address issues related to the drawing capab...,Redesign the hole pattern on the board to allo...,Employ CAD software to create a variety of hol...,Add a warning label to inform users of the enh...
72,Sensory Toys with Balls and Beads,Fact,The product has received multiple complaints r...,Reducing the Noise Level of the Magnetic Beads...,7,Incorporation of a Ball Release Mechanism,Prevent the magnetic balls from falling back i...,Incorporate a mechanism that pushes the balls ...,Mechanism Design: Use CAD software to create a...,Conduct a cost-benefit analysis to determine t...


In [33]:
weighted_trait_df_graph.head(3)

,data_label,positive_sentiment,negative_sentiment,rating_avg,rating,id,how_product_is_used,media,where_product_is_used,user_description,asin,observation_count,type,cluster_label,cluster_id_list
0,added more engaging features for adults,0.986908,0.002851,5,"[5, 5, 5]","[81, 186, 356]",['Entertaining toddlers'],[],[],['Parent of a toddler'],B07XCRT49W,3,Improvement,Product Improvements and Features,"[195, 356, 70, 136, 81, 241, 21, 121, 186]"
1,improved noise level and magnet strength,0.934750,0.004589,4,[4],[312],[],[],[],['Parent or caregiver of children who use the ...,B07XCRVK2Y,1,Improvement,Reduced Noise Levels,"[7, 267, 269, 400, 150, 300, 172, 312, 185, 19..."
2,pen was lost quickly,0.177518,0.497646,4,"[4, 4]","[352, 202]",[],[],[],['Parent or caregiver of children who used the...,B07XCRVK2Y,2,Issue,Pen Attachment and Loss,"[130, 391, 9, 138, 270, 23, 408, 156, 285, 31,..."


In [30]:
# Perform the merge based on matching columns
new_df = pd.merge(cluster_solutions_df[['type', 'cluster_label']], weighted_trait_df_graph[['type', 'cluster_label', 'cluster_id_list']], on=['type', 'cluster_label'], how='left')
new_df.drop_duplicates(subset = ['type', 'cluster_label'], inplace=True)
second_df = pd.merge(cluster_solutions_df, new_df, on=['type', 'cluster_label'], how='left')
cluster_solutions_df = second_df

cluster_solutions_df.rename(columns={"cluster_id_list": "id_of_reviews_inside_cluster"}, inplace=True)

In [36]:
cluster_solutions_df['number_of_observations'] = cluster_solutions_df['id_of_reviews_inside_cluster'].map(lambda x: len(x))

In [151]:
weighted_trait_df_graph

,data_label,positive_sentiment,negative_sentiment,rating_avg,rating,id,how_product_is_used,media,where_product_is_used,user_description,asin,observation_count,type,cluster_label,cluster_id_list
0,added more engaging features for adults,0.986908,0.002851,5,"[5, 5, 5]","[81, 186, 356]",[Entertaining toddlers],[],[],[Parent of a toddler],B07XCRT49W,3,Improvement,Product Improvements and Features,"[195, 356, 70, 136, 81, 241, 21, 121, 186]"
1,improved noise level and magnet strength,0.934750,0.004589,4,[4],[312],[],[],[],[Parent or caregiver of children who use the p...,B07XCRVK2Y,1,Improvement,Reduced Noise Levels,"[7, 267, 269, 400, 150, 300, 172, 312, 185, 19..."
2,pen was lost quickly,0.177518,0.497646,4,"[4, 4]","[352, 202]",[],[],[],[Parent or caregiver of children who used the ...,B07XCRVK2Y,2,Issue,Pen Attachment and Loss,"[130, 391, 9, 138, 270, 23, 408, 156, 285, 31,..."
3,losing the pen renders the toy useless,0.188561,0.561600,3,"[3, 3]","[208, 368]",[Entertaining toddlers during car rides],[],[In the car],[Parent of a toddler],B07XCRT49W,2,Issue,Pen Attachment and Loss,"[130, 391, 9, 138, 270, 23, 408, 156, 285, 31,..."
4,improved secure attachment for pen,0.979304,0.002445,4,"[4, 4]","[156, 316]",[In a classroom setting],[],[],[Teacher who uses the board in a classroom set...,B07XCRT49W,2,Improvement,Pen Attachment and Security,"[130, 138, 23, 408, 151, 156, 285, 31, 40, 173..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,magnetic pen attachment included,0.188789,0.463724,2,"[2, 2, 2]","[184, 344, 59]",[],[],[],[],B07XCRT49W,3,Fact,Magnetic Drawing Tools and Toys,"[130, 259, 132, 262, 7, 391, 138, 266, 396, 14..."
437,fixed issue with beads getting stuck,0.925880,0.002813,4,"[4, 4, 4]","[60, 375, 215]","[As a boredom buster for kids in the car, In t...",[],"[Car, In the car]",[Parent who uses the toy to keep their child e...,B07XCRT49W,3,Improvement,Product Quality and Customer Service,"[397, 276, 279, 280, 170, 310, 60, 208, 215, 9..."
438,slots for pushing balls with attached stylus,0.353720,0.198029,5,[5],[397],[Sensory play],[],[],[Parent of a child with sensory issues],B01M4OV4Q4,1,Fact,Sensory Toys with Balls and Beads,"[11, 397, 144, 401, 146, 400, 29, 296, 180, 18..."
439,added replacement balls or boards,0.124702,0.623844,3,"[3, 2, 3, 2, 3]","[238, 208, 368, 118, 93]","[Entertaining children during car rides, Playi...",[],[In the car],[Parent of a toddler],B07XCRVK2Y,5,Improvement,Product Quality and Customer Service,"[397, 276, 279, 280, 170, 310, 60, 208, 215, 9..."


In [198]:
for column_name in ['rating', 'id', 'how_product_is_used', 'media', 'where_product_is_used', 'user_description']:
    try:
        weighted_trait_df_graph[column_name] = weighted_trait_df_graph[column_name].map(eval)
    except:
        pass


grouped_df = weighted_trait_df_graph.groupby(['type','cluster_label', 'asin']).agg({
    'observation_count': 'sum',
    'positive_sentiment': 'mean',
    'negative_sentiment': 'mean',
    'rating':'sum',
    'id': 'sum',
    'how_product_is_used': 'sum',
    'media': 'sum',
    'where_product_is_used': 'sum',
    'user_description': 'sum',
    }).reset_index()

In [199]:
# Calculate the total number of observations for each ASIN
total_observations = grouped_df.groupby('asin')['observation_count'].transform('sum')

# Calculate the percentage of observations for each cluster label per ASIN
grouped_df['percentage_of_total_observations_per_asin'] = grouped_df['observation_count'] / total_observations * 100


In [200]:
grouped_df[grouped_df['asin'] == 'B07X7YFZWG']['percentage_of_total_observations_per_asin'].sum()

100.0

In [201]:
grouped_df

,type,cluster_label,asin,observation_count,positive_sentiment,negative_sentiment,rating,id,how_product_is_used,media,where_product_is_used,user_description,percentage_of_total_observations_per_asin
0,Fact,Beads and Pen Functionality,B07Q899BPB,1,0.006160,0.926367,[2],[284],[Drawing],[],[],[],1.265823
1,Fact,Beads and Pen Functionality,B07X7YFZWG,6,0.772754,0.068995,"[5, 5, 5, 5, 5, 5]","[50, 16, 151, 105, 390, 230]","[Artistic activities, As a toy for kids, As a ...",[https://m.media-amazon.com/images/I/71Hw6nsWf...,"[At home, on the go]","[Parent or caregiver of a young child, Parent ...",1.662050
2,Fact,Beads and Pen Functionality,B07XCRT49W,7,0.772754,0.068995,"[5, 5, 5, 5, 5, 5, 5]","[105, 390, 230, 16, 151, 335, 175]","[Creating pictures and dropping beads, As a to...",[https://m.media-amazon.com/images/I/71Hw6nsWf...,"[At home, on the go]","[Parent or caregiver of a young child, Parent ...",1.939058
3,Fact,Beads and Pen Functionality,B07XCRVK2Y,6,0.772754,0.068995,"[5, 5, 5, 5, 5, 5]","[301, 105, 390, 230, 335, 175]","[As a toy for kids, Creating pictures and drop...",[https://m.media-amazon.com/images/I/71Hw6nsWf...,"[At home, on the go]",[Parent or caregiver of a child who enjoys pla...,1.639344
4,Fact,Beads and Pen Functionality,B09R9MMW6J,1,0.036637,0.549738,[3],[242],[],[],[],[Parent who purchased the product for their da...,1.041667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Issue,Product Quality and Customer Experience,B07XCRVK2Y,64,0.510477,0.218627,"[5, 5, 5, 5, 5, 4, 4, 4, 1, 5, 5, 5, 5, 5, 3, ...","[105, 390, 230, 168, 308, 226, 101, 386, 348, ...","[Creating pictures and dropping beads, Child e...",[https://m.media-amazon.com/images/I/71Hw6nsWf...,"[Home, At home, on the go, In bed, In a bed or...","[Parent of an autistic child, Parent or caregi...",17.486339
131,Issue,Product Quality and Customer Experience,B07ZJ8HRN8,47,0.916667,0.024947,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[259, 4, 132, 134, 135, 387, 137, 389, 396, 39...","[To write braille and make pictures, Arts and ...",[https://m.media-amazon.com/images/I/61V3ELqqL...,"[In bed, In a bed or stationary location, Rest...","[Pediatric clinic worker, Parent of a child wi...",83.928571
132,Issue,Product Quality and Customer Experience,B085Q3TLF8,1,0.836876,0.008234,[4],[267],[As a toy],[],[],[],11.111111
133,Issue,Product Quality and Customer Experience,B085Q4W3WX,47,0.916667,0.024947,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[259, 4, 132, 134, 135, 387, 137, 389, 396, 39...","[To write braille and make pictures, Arts and ...",[https://m.media-amazon.com/images/I/61V3ELqqL...,"[In bed, In a bed or stationary location, Rest...","[Pediatric clinic worker, Parent of a child wi...",47.474747


In [202]:
for column_name in ['rating', 'id', 'how_product_is_used', 'media', 'where_product_is_used', 'user_description', 'asin']:
    try:
        grouped_df[column_name] = grouped_df[column_name].map(eval)
    except:
        pass

grouped_df['asin'] = grouped_df['asin'].apply(lambda elem: ast.literal_eval(elem) if isinstance(elem, str) and elem.startswith('[') else [elem] if isinstance(elem, str) else elem)

cluster_df = grouped_df.groupby(['type','cluster_label']).agg({
    'asin':  lambda x: sum((lst for lst in x), []),
    'observation_count': 'sum',
    'positive_sentiment': 'mean',
    'negative_sentiment': 'mean',
    'rating':'sum',
    'id': 'sum',
    'how_product_is_used': 'sum',
    'media': 'sum',
    'where_product_is_used': 'sum',
    'user_description': 'sum',
    }).reset_index()

In [203]:
# Calculate the total number of observations for each ASIN
total_observations = sum(cluster_df['observation_count'])
# Calculate the percentage of observations for each cluster label per ASIN
cluster_df['percentage_of_total_observations'] = cluster_df['observation_count'] / total_observations * 100

In [204]:
cluster_df.shape

(21, 13)

In [207]:
cluster_df.head(3)

,type,cluster_label,asin,observation_count,positive_sentiment,negative_sentiment,rating,id,how_product_is_used,media,where_product_is_used,user_description,percentage_of_total_observations
0,Fact,Beads and Pen Functionality,"[B07Q899BPB, B07X7YFZWG, B07XCRT49W, B07XCRVK2...",21,0.472212,0.336618,"[2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[284, 50, 16, 151, 105, 390, 230, 105, 390, 23...","[Drawing, Artistic activities, As a toy for ki...",[https://m.media-amazon.com/images/I/71Hw6nsWf...,"[At home, on the go, At home, on the go, At ho...","[Parent or caregiver of a young child, Parent ...",1.354839
1,Fact,Drawing Board with Pen,"[B01M4OV4Q4, B07X7YFZWG, B07XCRT49W, B07XCRVK2Y]",45,0.622252,0.244382,"[4, 3, 3, 3, 4, 4, 5, 5, 4, 4, 4, 2, 2, 2, 5, ...","[408, 208, 368, 93, 104, 389, 82, 357, 156, 31...","[Entertaining children during car rides, Enter...",[],"[In the car, Classroom, Classroom, In the car,...","[Parent of a toddler, Parent who purchased the...",2.903226
2,Fact,Drawing Tablet and Stylus,"[B07X7YFZWG, B07XCRT49W, B07XCRVK2Y, B07ZJ8HRN...",35,0.899606,0.030447,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[298, 148, 13, 337, 177, 52, 20, 135, 377, 62,...","[Creating patterns, As a travel toy, Drawing, ...",[],"[At home, in the car, on airplanes, During tra...","[Parent or caregiver of young children, Parent...",2.258065


In [216]:
cluster_solutions_df.head(3)

,cluster_label,type,cluster_problem_statement,cluster_solution_title,cluster,improvement_title,improvement_objective,improvement_solution,improvement_steps,improvement_considerations,id_of_reviews_inside_cluster,number_of_observations
0,Noisy Operation,Issue,"The product, a car toy for kids, has been repo...",Implementing Noise Reduction Technology,0,Noise Reduction and Sound Dampening,Reduce the noise level of the product to enhan...,Add a volume control feature to allow users to...,Volume Control: Add a control knob on the side...,Conduct a feasibility analysis for cost-effect...,"[1, 393, 15, 150, 166, 300, 306, 182, 185, 57,...",21
1,Magnetic Drawing Tools and Toys,Fact,"The product, a magnetic drawing board set, has...",Improving the Magnet Design for Better Stability,0,Noise Reduction and Sound Dampening,Reduce the noise level of the product to enhan...,Add a volume control feature to allow users to...,Volume Control: Add a control knob on the side...,Conduct a feasibility analysis for cost-effect...,"[130, 259, 132, 262, 7, 391, 138, 266, 396, 14...",60
2,Board Game Malfunctions,Issue,"The product, a magnetic drawing board set, has...",Improving the Quality and Durability of the Balls,0,Noise Reduction and Sound Dampening,Reduce the noise level of the product to enhan...,Add a volume control feature to allow users to...,Volume Control: Add a control knob on the side...,Conduct a feasibility analysis for cost-effect...,"[5, 7, 8, 141, 26, 170, 172, 173, 310, 312, 31...",20


In [213]:
cluster_solutions_df.columns

Index(['cluster_label', 'type', 'cluster_problem_statement',
       'cluster_solution_title', 'cluster', 'improvement_title',
       'improvement_objective', 'improvement_solution', 'improvement_steps',
       'improvement_considerations', 'id_of_reviews_inside_cluster',
       'number_of_observations'],
      dtype='object')

In [226]:
total_observations

1550

In [225]:
len(list(set(cluster_solutions_df['improvement_title'])))

20

In [230]:
cluster_solutions_df[cluster_solutions_df['cluster'] == 9]

,cluster_label,type,cluster_problem_statement,cluster_solution_title,cluster,improvement_title,improvement_objective,improvement_solution,improvement_steps,improvement_considerations,id_of_reviews_inside_cluster,number_of_observations
39,Product Quality and Customer Service,Improvement,"The product, a magnetic drawing board set, has...",Adding Replacement Balls or Boards to the Prod...,9,Replacement or Refund Policy for Defective Pro...,Address the issue of defective products and en...,Add a replacement or refund option to the prod...,Replacement or Refund Option: Establish a poli...,Ensure the replacement or refund policy is fea...,"[397, 276, 279, 280, 170, 310, 60, 208, 215, 9...",19
40,Quality of Ball Pit Balls,Fact,"The product, a magnetic drawing board set, has...",Improving the Quality of the Magnetic Balls,9,Replacement or Refund Policy for Defective Pro...,Address the issue of defective products and en...,Add a replacement or refund option to the prod...,Replacement or Refund Option: Establish a poli...,Ensure the replacement or refund policy is fea...,"[8, 360, 173, 238, 244, 276, 118, 85, 313, 124...",11
80,Product Quality and Customer Service,Improvement,"The product, a magnetic drawing board set, has...",Adding Replacement Balls or Boards to the Prod...,9,Improved Quality Control for Defect Prevention,Prevent the recurrence of reported defects and...,Establish a replacement policy for defective p...,Replacement Policy: Establish a policy that al...,Ensure the replacement policy is feasible and ...,"[397, 276, 279, 280, 170, 310, 60, 208, 215, 9...",19
81,Quality of Ball Pit Balls,Fact,"The product, a magnetic drawing board set, has...",Improving the Quality of the Magnetic Balls,9,Improved Quality Control for Defect Prevention,Prevent the recurrence of reported defects and...,Establish a replacement policy for defective p...,Replacement Policy: Establish a policy that al...,Ensure the replacement policy is feasible and ...,"[8, 360, 173, 238, 244, 276, 118, 85, 313, 124...",11
